In [ ]:
! pip install PyPDF2 reportlab pdf2image img2pdf pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 83.1 MB/s eta 0:00:00
  Created wheel for img2pdf: filename=img2pdf-0.6.1-py3-none-any.whl size=51001 sha256=e83af91b9ba875f0e668bce0f0a1df04a582e77df8ec5547e4ef848c865bde7e
  Stored in directory: /root/.cache/pip/wheels/21/0a/64/6a8143672bcea0d35b8cf705ba12b11bb191367cfde88d4f35
Successfully built img2pdf


In [ ]:
!apt update
!apt install -y poppler-utils

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,804 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,752 kB]
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,092 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:13 https://ppa.launchpad

In [ ]:
import io
import os
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.colors import Color
from PyPDF2 import PdfReader, PdfWriter
from pdf2image import convert_from_path
from PIL import Image

In [ ]:
# Generate multiple repeating, diagonal French watermarked PDFs (translucent + line feeds + no overlap)
def create_watermark(text, output_path, page_size=letter):
    packet = io.BytesIO()
    can = canvas.Canvas(packet, pagesize=page_size)

    # Translucent Gray
    transparent_gray = Color(0.5, 0.5, 0.5, alpha=0.5)
    can.setFillColor(transparent_gray)
    can.setFont("Helvetica-Bold", 25)  # The font size can be made larger to emphasize the watermark

    can.saveState()

    # coordinate system of rotation
    can.translate(page_size[0] / 2, page_size[1] / 2)
    can.rotate(45)

    # Calculate watermark text width
    text_width = can.stringWidth(text, "Helvetica-Bold", 30)

    # Define horizontal and vertical step lengths (intervals)
    x_start = -2 * page_size[0]  # Guaranteed watermark coverage on all pages
    x_end = 2 * page_size[0]
    y_start = -2 * page_size[1]
    y_end = 2 * page_size[1]

    x_step = text_width + 150  # Horizontal spacing to ensure that text does not overlap
    y_step = 200  # Vertical spacing, on demand

    # Cyclic tiling watermark
    for x in range(int(x_start), int(x_end), int(x_step)):
        for y in range(int(y_start), int(y_end), int(y_step)):
            can.drawString(x, y, text)

    can.restoreState()
    can.save()
    packet.seek(0)

    with open(output_path, 'wb') as f:
        f.write(packet.read())

# Overlay watermark on PDF
def add_watermark(input_pdf, output_pdf, watermark_pdf):
    reader = PdfReader(input_pdf)
    writer = PdfWriter()
    watermark = PdfReader(watermark_pdf).pages[0]

    for page in reader.pages:
        page.merge_page(watermark)
        writer.add_page(page)

    with open(output_pdf, 'wb') as f:
        writer.write(f)

# Turn the picture and then generate image-based PDF (with Pillow, does not rely on img2pdf)
def pdf_to_image_pdf(input_pdf, output_pdf, dpi=200):
    images = convert_from_path(input_pdf, dpi=dpi)

    rgb_images = []
    for img in images:
        if img.mode != 'RGB':
            img = img.convert('RGB')
        rgb_images.append(img)

    rgb_images[0].save(output_pdf, save_all=True, append_images=rgb_images[1:])

# -------------------
# The following is a sample call code, the path into your own PDF file path
original_pdf = "/content/2025-05-file.pdf"
watermark_pdf = "/content/watermark.pdf"
watermarked_pdf = "/content/watermarked.pdf"
final_image_pdf = "/content/Final_watermarked_version.pdf"

# 1. 生成带换行透明水印PDF
create_watermark("Ce document est fourni uniquement pour la location", watermark_pdf)

# 2. 叠加水印到原PDF
add_watermark(original_pdf, watermarked_pdf, watermark_pdf)

# 3. 转成图片型PDF，彻底防篡改
pdf_to_image_pdf(watermarked_pdf, final_image_pdf)

print("✅ Image type anti-tamper PDF has been generated, path: ", final_image_pdf)

✅ 图片型防篡改PDF已生成，路径： /content/最终加水印版本.pdf
